In [1]:
__all__ = ['NumericalSemigroup']  # So NumericalSemigroup is imported when "from <module> import *" is used
from collections import deque

class NumericalSemigroup:
    def __init__(self, generators):
        if not isinstance(generators, list):
            raise TypeError("Generators must be provided as a list.")
        self.gens = generators
        self.multiplicity = min(generators)
        self.gaps = self.compute_gaps()
        self.Frobenius = max(self.gaps) if self.gaps else None
        self.genus = len(self.gaps)

    def Apery_set(self, m):
        # Compute Apery set from generators
        # ONLY WORKS WHEN m IS A GENERATOR
        # returns Apery set as list
        gens = self.gens
        gensnoM = gens.copy()
        gensnoM.remove(m)

        C = (m-1)*(sum(gensnoM)) + 1
        
        Apery_set = [C] * m
        Apery_set[0] = 0
        
        Q = deque([0])

        while Q:
            n = Q.popleft()
            x = n % m
            if n <= Apery_set[x]:
                for g in gensnoM:
                    y = n + g
                    z = y % m
                    if y < Apery_set[z]:
                        Apery_set[z] = y
                        Q.append(y)
        Apery_set.remove(0)
        return Apery_set

    def compute_gaps(self):
        m = self.multiplicity
        Ap = self.Apery_set(m)
        gaps = []
        for i in Ap:
            while i > m:
                i = i - m
                gaps.append(i)
        gaps = sorted(set(gaps))
        return gaps
        
    def Kunz_Coords(self):
        m = self.multiplicity
        Ap = self.Apery_set(m)
        lAp = list(range(len(Ap)))
        Kunz = []
        for i in lAp:
            Kunz.append((Ap[i] - (i + 1)) / m)
        return Kunz
        
    def Remove_Min_Gen(self, r):
        # r is the mingen you want to remove
        Gens = self.gens.copy()
        Gens.remove(r)
        m = self.multiplicity
        
        if r == m:
            Gens.append(2 * r)
            Gens.append(2 * r + 1)
            return Gens
            
        k = m + r
        genslessr = [i for i in Gens if i < r]

        queue = deque(genslessr)
        while queue:
            n = queue.popleft()
            for g in genslessr:
                y = g + n
                if y < k:
                    queue.append(y)
                if y == k:
                    return Gens
        Gens.append(k)
        return Gens
        
    def Children(self):
        #takes numerical semigroup
        #returns its children as a list of numerical semigroups
        Gens = self.gens
        F = self.Frobenius
        C = []  # create list of children
        for i in Gens:
            if F < i:  # if i is effective gen
                newGens = self.Remove_Min_Gen(i)
                C.append(NumericalSemigroup(newGens))
        return C 

In [2]:
gens = [4,7,9]
sgp = NumericalSemigroup(gens)
print(f'Frobenius: {sgp.Frobenius} ')

Frobenius: 10 


In [3]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []
        
    def add_child(self, child_node):
        self.children.append(child_node)
        
class Tree:
    def __init__(self, root_value):
        self.root = Node(root_value)

    def find(self, value, node=None):
        if node is None:
            node = self.root
        if node.value == value:
            return node
        for child in node.children:
            result = self.find(value, child)
            if result is not None:
                return result
        return None

    def insert(self, parent_value, child_value):
        parent_node = self.find(parent_value)
        if parent_node is None:
            raise ValueError(f'Parent node with value {parent_value} not found.')
        child_node = Node(child_value)
        parent_node.add_child(child_node)

    def display(self, node=None, prefix="", is_last=True):
        if node is None:
            node = self.root
        print(prefix + ("└── " if is_last else "├── ") + str(node.value))
        prefix += "    " if is_last else "│   "
        child_count = len(node.children)
        for i, child in enumerate(node.children):
            is_last_child = i == (child_count - 1)
            self.display(child, prefix, is_last_child)

In [4]:
from collections import deque 
from collections import Counter

def branchmult(MAXG, m, S): #prints elements with multiplicity m in the num sgp tree starting at S
    tree = Tree(S.gens) #starts tree with generators of S as root 
    queue = deque([tree.root])
    g = S.genus
    numClst = []
    
    with open('/home/samwisega/MultChild.csv', 'w') as file:
        file.write('Genus,Generators,Number of Children, Kunz Coords\n')
        while queue and g <= MAXG:
            node = queue.popleft()
            current_gen = NumericalSemigroup(node.value)
            f = current_gen.genus
            C = current_gen.Children()
            numC = len(C)
            numClst.append(numC)
            
            # for current sgp print genus, min gens and # of children  
            # to a csv file 
            file.write(f'{g},"{node.value}",{numC},"{current_gen.Kunz_Coords()}"\n')

            for c in C: #for each num sgp c, in the list of children C of S
                tree.insert(node.value, c.gens)  
                # Insert the new semigroup into the tree
                if m in c.gens: #if c has multiplicity m
                    queue.append(Node(c.gens))  
                    # Append the new node to the queue 
           
            nextnode = queue[0]
            g = NumericalSemigroup(nextnode.value).genus
            
            if f < g: 
                print('genus', f, Counter(numClst))
                numClst = []

In [5]:
def branchmultFrob(MAXG, m, S): #prints elements with multiplicity m in the num sgp tree starting at S
    tree = Tree(S.gens) #starts tree with generators of S as root 
    queue = deque([tree.root])
    g = S.genus
    num_1_mod4 = []
    num_2_mod4 = []
    num_3_mod4 = []
    
    with open('/home/samwisega/MultChild.csv', 'w') as file:
        file.write('Genus, i, # of Sgps w/ F = i mod 4, # w/ 0 child, # w/ 1 child, # w/ 2 child, # w/ 3 child\n')
        while queue and g <= MAXG:
            node = queue.popleft()
            current_gen = NumericalSemigroup(node.value)
            h = current_gen.genus 
            F = current_gen.Frobenius
            C = current_gen.Children()
            numC = len(C)
            a = F % m 
            
            if a == 1: 
                num_1_mod4.append(numC)

            if a == 2: 
                num_2_mod4.append(numC)

            if a == 3: 
                num_3_mod4.append(numC)
            
            for c in C: #for each num sgp c, in the list of children C of S
                tree.insert(node.value, c.gens)  
                # Insert the new semigroup into the tree
                if m in c.gens: #if c has multiplicity m
                    queue.append(Node(c.gens))  
                    # Append the new node to the queue 

            nextnode = queue[0]
            g = NumericalSemigroup(nextnode.value).genus
            
            if h < g: 
                print(h)
                file.write(f'{h},{1},"{len(num_1_mod4)}","{num_1_mod4.count(0)}","{num_1_mod4.count(1)}","{num_1_mod4.count(2)}","{num_1_mod4.count(3)}"\n')
                file.write(f'{h},{2},"{len(num_2_mod4)}","{num_2_mod4.count(0)}","{num_2_mod4.count(1)}","{num_2_mod4.count(2)}","{num_2_mod4.count(3)}"\n')
                file.write(f'{h},{3},"{len(num_3_mod4)}","{num_3_mod4.count(0)}","{num_3_mod4.count(1)}","{num_3_mod4.count(2)}","{num_3_mod4.count(3)}"\n')
                num_1_mod4 = []
                num_2_mod4 = []
                num_3_mod4 = []
                

In [6]:
def branchmultChild(MAXG, m, S): #prints elements with multiplicity m in the num sgp tree starting at S
    tree = Tree(S.gens) #starts tree with generators of S as root 
    queue = deque([tree.root])
    g = S.genus
    numClst = []
    
    with open('MultChild.csv', 'w') as file:
        file.write('Genus, # of Sgps, # w/ 0 child, # w/ 2 child, # w/ 3 child\n')
        while queue and g <= MAXG:
            node = queue.popleft()
            current_gen = NumericalSemigroup(node.value)
            h = current_gen.genus 
            C = current_gen.Children()
            numC = len(C)
            numClst.append(numC)
            
            for c in C: #for each num sgp c, in the list of children C of S
                tree.insert(node.value, c.gens)  
                # Insert the new semigroup into the tree
                if m in c.gens: #if c has multiplicity m
                    queue.append(Node(c.gens))  
                    # Append the new node to the queue 

            nextnode = queue[0]
            g = NumericalSemigroup(nextnode.value).genus
            
            if h < g: 
                print(h)
                file.write(f'{h},"{len(numClst)}","{numClst.count(0)}","{numClst.count(2)}","{numClst.count(3)}"\n')
                numClst = []
                

In [ ]:
S = NumericalSemigroup([4,5,6,7])
branchmultChild(100,4,S)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
